<a href="https://colab.research.google.com/github/edsondamasceno/deep-learning-methods/blob/main/Hyperparameters_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import keras
import numpy as np

# Load data

In [ ]:
train_path = '/content/drive/MyDrive/CT-COVID-19/Train'
test_path = '/content/drive/MyDrive/CT-COVID-19/Test'
val_path = '/content/drive/MyDrive/CT-COVID-19/Val'

In [ ]:
train_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(train_path,
                                                                       target_size=(224,224),
                                                                       color_mode='rgb',
                                                                       batch_size=int(64), #64
                                                                       shuffle=True)
test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
                                                                      target_size=(224,224),
                                                                      color_mode='rgb',
                                                                      batch_size=int(64), #64
                                                                      shuffle=True)
val_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
                                                                     target_size=(224,224),
                                                                     color_mode='rgb',
                                                                     batch_size=int(64), #64
                                                                     shuffle=True)

Found 422 images belonging to 2 classes.
Found 214 images belonging to 2 classes.
Found 86 images belonging to 2 classes.


# Hyperopt

In [ ]:
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
space = { 'learning_rate': hp.uniform('learning_rate', 0, 1),
          'dropout01': hp.uniform('dropout01', 0.1, 0.6),
          'dropout02': hp.uniform('dropout02', 0.1, 0.6),
          'dropout03': hp.uniform('dropout03', 0.1, 0.6),
          'dropout04': hp.uniform('dropout04', 0.1, 0.6),
          'fcl01': hp.uniform('fcl01', 200, 300),
          'fcl02': hp.uniform('fcl02', 50, 150)
        }

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras import optimizers

def hyperparameters(space):
    np.random.seed(1337)          # for reproducibility
    model = Sequential()

    #1st conv layer
    model.add(Conv2D(32, (3,3), padding="same",
                    input_shape=(224, 224, 3),data_format="channels_last"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(space['dropout01']))

    #2nd conv layer
    model.add(Conv2D(32, (3,3), padding="same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(space['dropout02']))

    #3rd conv layer
    model.add(Conv2D(32, (3,3), padding="same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(space['dropout03']))

    #4th conv layer
    model.add(Conv2D(32, (3,3), padding="same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(space['dropout04']))

    model.add(Flatten())

    model.add(Dense(space['fcl01']))
    model.add(Dense(space['fcl02'],name ='feature_dense'))

    #output FC
    model.add(Dense(2))
    model.add(Activation('sigmoid'))
    adam = optimizers.Adam(learning_rate=space['learning_rate'], beta_1=0.9, beta_2=0.999)
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
    #model.summary()

    history = model.fit_generator(train_batches,epochs=50,validation_data=val_batches,verbose=1)

    predicts_vectors = model.predict_generator(test_batches, verbose=1)
    pred = np.argmax(predicts_vectors,axis=1)

    from sklearn.metrics import accuracy_score
    from sklearn.preprocessing import LabelBinarizer

    lb = LabelBinarizer ()
    lb.fit(test_batches.labels)
    y_test = lb.transform(test_batches.labels)
    y_pred = lb.transform(pred)
    acc = accuracy_score(y_pred, y_test)
    print('Accuracy:', acc)
    #sys.stdout.flush()
    return {'loss': acc, 'status': STATUS_OK}


In [ ]:
from hyperopt import rand
trials = Trials()
best = fmin(fn=hyperparameters,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

In [ ]:
print(best)